In [1]:
from transformers import AutoModelForCausalLM
from copy import deepcopy

model_id = 'facebook/opt-125m'
model = AutoModelForCausalLM.from_pretrained(model_id)
cp_model = deepcopy(model)

/home/jwjeong/anaconda3/envs/quant/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jwjeong/anaconda3/envs/quant/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
import torch
def l2_norm(x):
    return torch.linalg.matrix_norm(x)

In [3]:
import numpy as np
def QSNR(tensor1, tensor2):
    ratio = l2_norm(tensor1-tensor2) / l2_norm(tensor2)
    return -10*torch.log(ratio)
    

In [9]:
from quant import quant_unpack, dequant_unpack, quant, dequant
bits = 4
gs = 128
scale, zero, qs = quant(bits, gs, cp_model)
q_x = dequant(scale, zero, qs, gs, bits)
QSNRs = {}
for key in q_x.keys():
    if key.split('.')[-1] != 'lm_head':
        if key.split('.')[-1] == 'fc1':
            weight = key+'.weight'
            QSNRs[key] = round(QSNR(q_x[key], cp_model.state_dict()[weight]).item(), 3)

In [10]:
QSNRs

{'model.decoder.layers.0.fc1': 23.362,
 'model.decoder.layers.1.fc1': 22.23,
 'model.decoder.layers.2.fc1': 21.902,
 'model.decoder.layers.3.fc1': 22.178,
 'model.decoder.layers.4.fc1': 22.654,
 'model.decoder.layers.5.fc1': 22.833,
 'model.decoder.layers.6.fc1': 22.797,
 'model.decoder.layers.7.fc1': 22.787,
 'model.decoder.layers.8.fc1': 22.845,
 'model.decoder.layers.9.fc1': 22.924,
 'model.decoder.layers.10.fc1': 22.972,
 'model.decoder.layers.11.fc1': 22.594}